In [1]:
conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = '0_1SPLIT_1413851857_sessionsession_1413851857'
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = 's3n://contentwise-research-poli/0_1SPLIT_1413851857_sessionsession_1413851857'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = 'contentwise-research-poli'
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 5
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = "split2.split"
conf['general']['bucketName'] = 'contentwise-research-poli'
conf['general']['tmpDir'] = "/tmp"

conf['algo'] = {}
conf['algo']['name'] = 'ImplicitPlaylistFallBack'



In [2]:
import matplotlib.pyplot as plt
from matplotlib import cm

def bar_plot(x, y, labels, metric):
    n_groups = len(x)
    
    fig, ax = plt.subplots()
    
    index = np.arange(n_groups)
    bar_width = float(1)/len(y)
    
    opacity = 0.4
    
    for idx, label_y in enumerate(zip(labels, y)):
        label, values = label_y
        plt.bar(index + idx*bar_width, values, bar_width,
                alpha=opacity,
                color=cm.jet(float(idx)/len(y)),
                label=label)

    plt.xlabel('N')
    plt.ylabel(metric)
    plt.xticks(index, x)
    #plt.legend(loc=2)
    
    plt.tight_layout()
    plt.show()


In [3]:
import boto
import json
import numpy as np
from os import path

s3 = boto.connect_s3()
mybucket = s3.get_bucket(conf['general']['bucketName'])
base_path = path.join('s3n://', conf['general']['bucketName'])
rec_path = path.join(conf['general']['clientname'], conf['split']["name"], 'Rec')
algo_path = path.join(rec_path, conf['algo']['name'])
print algo_path

split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/ImplicitPlaylistFallBack


In [4]:
# list all /metrics folders for the given algorithm
metrics_end = path.join(conf['evaluation']['name'], 'metrics', '_SUCCESS')
_success_len = len('_SUCCESS')
metric_folders = [k.name[:-_success_len] for k in mybucket.list(prefix=algo_path) if k.name.endswith(metrics_end)]

In [5]:
print metric_folders[:10]

[u'split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/ImplicitPlaylistFallBack_shk_10_clustSim_0100_decay_0100_numGH100skipTh0#10#SAGH#01#01/recall@N/metrics/', u'split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/ImplicitPlaylistFallBack_shk_10_clustSim_0100_decay_0300_numGH100skipTh0#10#SAGH#03#01/recall@N/metrics/', u'split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/ImplicitPlaylistFallBack_shk_10_clustSim_0100_decay_0500_numGH100skipTh0#10#SAGH#05#01/recall@N/metrics/', u'split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/ImplicitPlaylistFallBack_shk_10_clustSim_0100_decay_0700_numGH100skipTh0#10#SAGH#07#01/recall@N/metrics/', u'split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/ImplicitPlaylistFallBack_shk_10_clustSim_0100_decay_0900_numGH100skipTh0#10#SAGH#09#01/recall@N/metrics/', u'split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/ImplicitPlaylistFallBack_shk_10_clustSim_0300_decay_0100_numGH100skipTh0#1

In [6]:
len(metric_folders)

147

In [ ]:
metrics = [
    (mf.split('/')[-4],
    sc.textFile(path.join(base_path, mf))\
    .map(lambda x : json.loads(x))\
    .map(lambda x: x['evaluation']['value'])\
    .collect()
    )
    for mf in metric_folders
    ]

In [ ]:
print metrics

In [ ]:
labels, y = zip(*metrics)

In [ ]:
N=3
x_up_to_N = conf['evaluation']['metric']['prop']['N'][:N]
y_up_to_N = [v[:N] for v in y]
x_up_to_N

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 16, 12  # that's default image size for this interactive session
bar_plot(x_up_to_N, y_up_to_N, labels, conf['evaluation']['name'])